In [1]:
import numpy as np
import xarray as xr

from mom6_tools.moc import MOCpsi
from mom6_tools.m6toolbox import genBasinMasks 

import matplotlib
import matplotlib.pyplot as plt

import warnings, yaml, os

warnings.filterwarnings("ignore")

/glade/work/jiangzhu/conda-envs/my-npl-2024a/lib/python3.11/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'allowed-failures' has been deprecated; please use 'distributed.scheduler.allowed-failures' instead
  warnings.warn(


Basemap module not found. Some regional plots may not function properly


In [2]:
archv_dir = '/glade/derecho/scratch/jiangzhu/archive'
hist_dir = 'ocn/hist'
casename = 'b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02'

work_dir = f"{archv_dir}/{casename}/{hist_dir}/"
print(work_dir)
ystart = 1
yend = 98

/glade/derecho/scratch/jiangzhu/archive/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02/ocn/hist/


# MOM6 Grid info

In [3]:
mom6_grid_ds = xr.open_dataset('/glade/work/jiangzhu/data/inputdata/mom/tx2_3v2/ocean_topog_230413.nc')

mom6_grid_ds['basin_mask'] = mom6_grid_ds.mask.copy(deep=True)
mom6_grid_ds

<xarray.Dataset>
Dimensions:     (lonh: 540, lath: 480, lonq: 541, latq: 481)
Coordinates:
  * lonh        (lonh) float64 -286.7 -286.0 -285.3 -284.7 ... 71.33 72.0 72.67
  * lath        (lath) float64 -81.56 -81.46 -81.36 -81.26 ... 89.33 89.6 89.86
  * lonq        (lonq) float64 -287.0 -286.3 -285.7 -285.0 ... 71.67 72.33 73.0
  * latq        (latq) float64 -81.61 -81.51 -81.41 -81.31 ... 89.46 89.72 89.91
Data variables:
    geolon      (lath, lonh) float64 ...
    geolat      (lath, lonh) float64 ...
    geolonb     (latq, lonq) float64 ...
    geolatb     (latq, lonq) float64 ...
    mask        (lath, lonh) int32 ...
    depth       (lath, lonh) float32 ...
    basin_mask  (lath, lonh) int32 ...
Attributes:
    Description:                       Ocean Topography on MOM6 Grid
    Creator:                           gmarques
    Created:                           20230413
    Generating Code:                   interp_smooth.f90
    Model Grid Version:                tx2_3v2
    Source Topography Data:            /glade/work/bryan/Observations/Topogra...
    Source Mask Data:                  topo.sub150.tx2_3v2.srtm.edit4.nc
    Smoothing Scale L (grid lengths):  1.0
    Smoothing Function:                (L^2 - r^2)/(L^2 + r^2)
    Minimum Depth (m):                 10.0

In [4]:
basin_code = genBasinMasks(mom6_grid_ds.geolon.values,
                           mom6_grid_ds.geolat.values,
                           mom6_grid_ds.depth.values, verbose=False)

In [5]:
mom6_grid_ds['basin_mask'].data = basin_code
mom6_grid_ds.basin_mask

<xarray.DataArray 'basin_mask' (lath: 480, lonh: 540)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
  * lonh     (lonh) float64 -286.7 -286.0 -285.3 -284.7 ... 71.33 72.0 72.67
  * lath     (lath) float64 -81.56 -81.46 -81.36 -81.26 ... 89.33 89.6 89.86
Attributes:
    longname:  0 if land, 1 if ocean at tracer points
    units:     nondimensional

In [6]:
basin_mask = mom6_grid_ds.basin_mask
basin_mask_alt = xr.where(((basin_mask == 2) |
                           (basin_mask == 4) |
                           (basin_mask == 6) |
                           (basin_mask == 7) |
                           (basin_mask == 8)), 1, 0)
basin_mask_alt.plot(size=2)

In [7]:
conversion_factor = 1.e-9
components = ['vmo', 'vhml', 'vhGM']

In [8]:
%%time

for i_year in np.arange(ystart, yend + 1, 1):

    for i_month in np.arange(1, 13, 1):

        file_in = f"{work_dir}{casename}.mom6.h.z.{i_year:04d}-{i_month:02d}.nc"
        file_ou = f"{work_dir}{casename}.mom6.h.moc.{i_year:04d}-{i_month:02d}.nc"
        print(file_in)

        ds = xr.open_dataset(file_in)

        gmoc = np.zeros((1, len(components), len(ds.z_l), len(ds.yq)))
        amoc = np.zeros((1, len(components), len(ds.z_l), len(ds.yq)))
        
        for idx, item in enumerate(components):
            vtmp = ds[item].fillna(0).values

            psi_tmp = MOCpsi(vtmp) * conversion_factor
            gmoc[0, idx, :, :] = 0.5 * (psi_tmp[:, 0:-1, :] + psi_tmp[:, 1::, :])
            
            psi_tmp = MOCpsi(vtmp, vmsk=basin_mask_alt.values) * conversion_factor
            amoc[0, idx, :, :] = 0.5 * (psi_tmp[:, 0:-1, :] + psi_tmp[:, 1::, :])

        ds_ou = xr.Dataset(data_vars={'gmoc': (('time', 'components', 'z_l','yq'), gmoc),
                                      'amoc': (('time', 'components', 'z_l','yq'), amoc)},
                           coords={'time': ds.time, 'components': np.array([0, 1, 2]), 'z_l': ds.z_l, 'yq': ds.yq},
                           attrs=dict(description='Meridonal Overturning Circulation '))
        
        ds_ou.gmoc.attrs = {'units': 'Sv',
                            'long_name': 'Global Ocean Meridional Overturning Circulation'}
        ds_ou.amoc.attrs = {'units': 'Sv',
                            'long_name': 'Atlantic Ocean Meridional Overturning Circulation'}
        ds_ou.components.attrs = {'units': 'None',
                                  'long_name': 'Transport components (0=total, 1=submeso-induced, 2=eddy-induced)'}

        ds_ou.to_netcdf(file_ou, unlimited_dims='time')

/glade/derecho/scratch/jiangzhu/archive/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02/ocn/hist/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02.mom6.h.z.0001-01.nc
/glade/derecho/scratch/jiangzhu/archive/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02/ocn/hist/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02.mom6.h.z.0001-02.nc
/glade/derecho/scratch/jiangzhu/archive/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02/ocn/hist/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02.mom6.h.z.0001-03.nc
/glade/derecho/scratch/jiangzhu/archive/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02/ocn/hist/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02.mom6.h.z.0001-04.nc
/glade/derecho/scratch/jiangzhu/archive/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02/ocn/hist/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02.mom6.h.z.0001-05.nc
/glade/derecho/scratch/jiangzhu/archive/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02/ocn/hist/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02.mom6.h.z.0001-06.nc
/glade/derecho/scratch/jiangzhu/archive/b.e23_beta17.BLT1850.ne30pg3_t232.21ka.02/